<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

In [2]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [3]:
# enddate = date.today()
# startdate = enddate - relativedelta(years=4)

# df = yf.download('HINDZINC.NS', start=startdate, end=enddate)


In [4]:
#ticker_yahoo = yf.Ticker('HINDZINC.NS')
# last_quote = ticker_yahoo.fast_info['last_price']
# MA200 = ticker_yahoo.fast_info['two_hundred_day_average']
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)
#print(tickers)

In [5]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [6]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')
  


def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    #print(f"{ticker} {info['last_price']} {info['year_high']}")
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ath_list))
    return results

results = paralel_func(ath_list)


In [7]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)
#ticker_list = ticker_list.loc[(ticker_list['Last_price'] < ticker_list['ATH'] * 0.95) & (ticker_list['Last_price'] > ticker_list['ATH'] * 0.9) & (ticker_list['Last_price'] > ticker_list['two_EMA'])]

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)
#ticker_list
ticker_list.to_csv('Watchlist.csv', index=False)


In [8]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today()

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]
#tickers
df = yf.download(tickers, start= startdate, end= enddate)


[*********************100%***********************]  101 of 101 completed


In [9]:
# df.ta.supertrend(high='High', low='Low', close='Close', length=10, multiplier=1, append=True)
# df.to_csv('data1.csv')
#tickers
#help(ta.supertrend)

In [40]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

# ax_df = ta.supertrend(close=df[('Close', 'ABBOTINDIA.NS')]
#                                                          , high=df[('High', 'ABBOTINDIA.NS')]
#                                                          , low=df[('Low', 'ABBOTINDIA.NS')]
#                                                          , length=10, multiplier=1)

# df_copy['SUPERTd_10_1', 'ABBOTINDIA.NS'] = ax_df['SUPERTd_10_1.0']
# df_copy

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()

for ticker in tickers:
  #df[('EMA_200',ticker)] = ta.ema(df.loc[:,('Close', ticker)], length=200)
  df1 = pd.concat([df1 ,ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)], axis=1)
  #df1.columns.set_names(ticker,level=1, inplace=True)
#   df2 = pd.concat([df2 ,ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)], axis=1)
#   df3 = pd.concat([df3 ,ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)], axis=1)

#   SUPERT_10_1 =  ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
#   SUPERT_11_2 =  ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
#   SUPERT_12_3 =  ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
#   ax_df[('SUPERTd_10_1',ticker)] = SUPERT_10_1['SUPERTd_10_1.0']
#   ax_df[('SUPERTd_11_2',ticker)] = SUPERT_11_2['SUPERTd_11_2.0']
#   ax_df[('SUPERTd_12_3',ticker)] = SUPERT_12_3['SUPERTd_12_3.0']

#   ax_df[('SUPERTd_10_1',ticker)] = SUPERT_10_1['SUPERTd_10_1.0']
#   ax_df[('SUPERTd_11_2',ticker)] = SUPERT_11_2['SUPERTd_11_2.0']
#   ax_df[('SUPERTd_12_3',ticker)] = SUPERT_12_3['SUPERTd_12_3.0']

df1
#ax_df.drop(['Adj Close','Open','Low','High','Volume'], axis=1)

,SUPERT_10_1.0,SUPERTd_10_1.0,SUPERTl_10_1.0,SUPERTs_10_1.0,SUPERT_10_1.0,SUPERTd_10_1.0,SUPERTl_10_1.0,SUPERTs_10_1.0,SUPERT_10_1.0,SUPERTd_10_1.0,...,SUPERTl_10_1.0,SUPERTs_10_1.0,SUPERT_10_1.0,SUPERTd_10_1.0,SUPERTl_10_1.0,SUPERTs_10_1.0,SUPERT_10_1.0,SUPERTd_10_1.0,SUPERTl_10_1.0,SUPERTs_10_1.0
Date,,,,,,,,,,,,,,,,,,,,,
2019-02-05 00:00:00+05:30,0.000000,1,NaN,NaN,0.000000,1,NaN,NaN,0.000000,1,...,NaN,NaN,0.000000,1,NaN,NaN,0.000000,1,NaN,NaN
2019-02-06 00:00:00+05:30,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,1,...,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN
2019-02-07 00:00:00+05:30,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,1,...,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN
2019-02-08 00:00:00+05:30,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,1,...,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN
2019-02-11 00:00:00+05:30,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,1,...,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-30 00:00:00+05:30,21512.247105,-1,NaN,21512.247105,332.607098,-1,NaN,332.607098,341.867856,1,...,NaN,743.536132,1586.546359,1,1586.546359,NaN,438.060605,-1,NaN,438.060605
2023-01-31 00:00:00+05:30,21383.235090,-1,NaN,21383.235090,315.518760,1,315.51876,NaN,359.721064,1,...,NaN,743.536132,1586.546359,1,1586.546359,NaN,438.060605,-1,NaN,438.060605
2023-02-01 00:00:00+05:30,21336.663632,-1,NaN,21336.663632,315.518760,1,315.51876,NaN,359.721064,1,...,NaN,743.536132,1586.546359,1,1586.546359,NaN,438.060605,-1,NaN,438.060605
